### General Steps (Applies to All Models)

Load a pretrained model

Remove or ignore the original classifier

Add your own layers

Freeze base layers (initially)

Train on your dataset

(Optional) Fine-tune deeper layers **bold text** bold text

In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models


In [34]:
train_data = keras.preprocessing.image_dataset_from_directory(
    "dataset/train",
    image_size=(224, 224),
    batch_size=32
)

val_data = keras.preprocessing.image_dataset_from_directory(
    "dataset/val",
    image_size=(224, 224),
    batch_size=32
)


In [ ]:
normalization_layer = layers.Rescaling(1./255)

train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))


### Load Pretrained Model (WITHOUT top layer)

In [ ]:
base_model = keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)


### Freeze the Pretrained Layers (FEATURE EXTRACTION)

In [ ]:
base_model.trainable = False # Freeze the Base Model


### Build New Model on Top

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")  # binary classification
])


In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"])
#  At this stage:

# Base CNN = frozen

# Only new layers are trained


In [ ]:
history = model.fit(
    train_data,
    epochs=5,
    validation_data=val_data
)


In [ ]:
model.evaluate(val_data)


### OPTIONAL: Fine-Tuning (Advanced)

##### After initial training, you can unfreeze some layers.

In [ ]:
# Unfreeze top layers only
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5), # Use very low learning rate.
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
model.fit(
    train_data,
    epochs=5,
    validation_data=val_data
)


In [ ]:
model.evaluate(val_data)


In [ ]:
model.save("final_cnn_model.h5")
